In [67]:
#encoding = utf-8
import numpy as np
import pandas as pd
import operator

def CalcGiNiIndex(DataSet) :
    Num_length = len(DataSet)
    labelcounts = {}
    for feature in DataSet :
        currentlabel = feature[-1]
        #用字典统计类别及其数目
        if currentlabel not in labelcounts.keys() :
            labelcounts[currentlabel] = 1
        else :
            labelcounts[currentlabel] += 1
    GiNi_index = 0
    for key in labelcounts.keys() :
        #二分类求基尼指数：GiNi = 2 * p * （1 - p）
        GiNi_index = 2 * (float(labelcounts[key]) / Num_length) * (1 - float(labelcounts[key]) / Num_length)
    return GiNi_index

def createDataSet() :
    DataSet = [['晴天', '炎热', '高', '弱风', '否'],
                 ['晴天', '炎热', '高', '强风', '否'],
                 ['阴天', '炎热', '高', '弱风', '是'],
                 ['雨天', '温', '高', '弱风', '是'],
                 ['雨天', '冷', '中', '弱风', '是'],
                 ['雨天', '冷', '中', '强风', '否'],
                 ['阴天', '冷', '中', '强风', '是'],
                 ['晴天', '温', '高', '弱风', '否'],
                 ['晴天', '冷', '中', '弱风', '是'],
                 ['雨天', '温', '中', '弱风', '是'],
                 ['晴天', '温', '中', '强风', '是'],
                 ['阴天', '温', '高', '强风', '是'],
                 ['阴天', '炎热', '中', '弱风', '是'],
                 ['雨天', '温', '高', '强风', '否']]
#     file = pd.read_excel('Data.xlsx')
#     DataSet = file.iloc[ : , 1 : ]
#     DataSet = np.array(DataSet)
    labels = ['天气', '温度', '湿度', '风力']
    
    return DataSet, labels

def splitDataSet(DataSet, axis, value): 
    #计算以某个特征分类后剩下的数据量。
    #axis表示第i个特征，value表示在改特征的情况下的具体表现，如天气特征有雨天。阴天等。
    retDataSet = []
    #创建一个新列表，准备提取数据
    for featVec in DataSet :
        #剔除在数据集中需要被分类的特征的列行
        if featVec[axis] == value :
            reducedFeatVec = featVec[ : axis]
            reducedFeatVec.extend(featVec[axis + 1 : ])
            retDataSet.append(reducedFeatVec)
    return retDataSet

def ChooseBestFeatureToSplit(DataSet) :
    #计算父亲节点的GiNi指数
    FatherGiNi = CalcGiNiIndex(DataSet)
    BestIoFoGain = 0
    BestFeature = -1
    numFeature = len(DataSet[0]) - 1
    #去掉最后一列
    for i in range(numFeature) :
        #每个特征下的具体表现形式
        featList = [example[i] for example in DataSet]
        uniqueVals = set(featList)#去重复
        newGiNi = 0
        for value in uniqueVals :
            #计算每一种表现形式GiNi指数的权重和
            subData = splitDataSet(DataSet, i, value)
            prob = len(subData) / float(len(DataSet))
            newGiNi += prob * CalcGiNiIndex(subData)
        infogain = FatherGiNi - newGiNi#计算信息增益
        if infogain > BestIoFoGain :
            #比较信息增益的大小，更新最佳分类的特征
            BestIoFoGain = infogain
            BestFeature = i
    return BestFeature

def majorityCnt(classList):    #按分类后类别数量排序，比如：最后分类为2yes1no，则判定为yes；
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    #字典逆序排序
    sortedClassCount = sorted(classCount.items(), key = lambda x : x[1], reverse=True)
    return sortedClassCount[0][0]

def createTree(DataSet, labels) :
    ConditionList = [example[-1] for example in DataSet]
    #list.count(element) 方法用于统计某个元素在列表中出现的次数。
    if ConditionList.count(ConditionList[0]) == len(ConditionList) :#yes or no
        return ConditionList[0]
    #dataSet[0]取矩阵第一行，dataSet[0][0]取矩阵第一行第一列元素
    #递归终止条件2：使用完所有特征，则返回最后出现次数最多的那个标签
    if len( DataSet[0] ) == 1:   
        return majorityCnt(classList)
    
     #以上两个终止条件都不满足，开始选择最优特征划分，已经有了一个方框，准备往方框中写入判断问题
    BestFeature = ChooseBestFeatureToSplit(DataSet)
    BestFeatLabel = labels[BestFeature]
    mytree = {BestFeatLabel : {}}
    #用过了该特征，将该特征从所有特征列表中删除
    del (labels[BestFeature])
    featValues = [example[BestFeature] for example in DataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[ : ]
        splitdata = splitDataSet(DataSet, BestFeature, value)
        #递归
        mytree[BestFeatLabel][value] = createTree(splitdata, subLabels)
    return mytree

DataSet, labels = createDataSet()# 创造示列数据
print(createTree(DataSet, labels))# 输出决策树模型结果


{'天气': {'阴天': '是', '晴天': {'湿度': {'中': '是', '高': '否'}}, '雨天': {'风力': {'弱风': '是', '强风': '否'}}}}
